jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
!pip install tk
!pip install rdp
!pip install pillow
!pip install numpy


In [ ]:
from tkinter import *
from tkinter import ttk
import time, threading
from PIL import Image, ImageDraw
import numpy as np

screenx, screeny, offsetx, offsety = 1280, 720 , 360, 80
# screenx, screeny, offsetx, offsety = 256, 256 , 160, 90
canvasImage = Image.new("L", (screenx- offsetx * 2, screeny- offsety* 2), 255)
draw = ImageDraw.Draw(canvasImage)

In [ ]:
#import tkMessageBox

#Globals
lastx, lasty = 0, 0
drawingStroke = False
startTime, promptTime, promptStartTime =  0, 60, 0
strokeData, currentStroke = [], []
prompt = "A house is on fire, what would be useful to take out the fire?"
prompts = ["You're about to leave to lecture, what should you have with you?",
           "A house is on fire, what would be useful to take out the fire?",
           "You got lost in the woods at night, what items would be helpful to have?",
           "Prompt example 4",
           "Prompt example 5"]
# answers to prompt, in same order
promptAnswers = [["calculator", "computer", "backpack", "laptop", "hand", "compass", "car", "bicycle", "eyeglasses", "eye", "pencil", "square"],
                 ["hose", "firetruck", "broom", 'garden hose', 'fire hydrant', 'flashlight', 'ladder', 'helmet', 'bucket'],             
                 ["garden hose",'flashlight'],
                 ["example1", "example1"],
                 ["example1", "example1"],]
promptIndex, correctAnswers = 0, []

#Definitions
def xy(event):
  global lastx, lasty, currentStroke, startTime
  lastx, lasty = event.x, event.y
  drawingStroke = True
  startTime = time.time()
  currentStroke = [[event.x],[event.y],[0]]

def addLine(event):
  global lastx, lasty, currentStroke, startTime
  distance = ((((event.x - lastx )**2) + ((event.y-lasty)**2) )**0.5)
  if distance < 1:
    return
  create_circle(event.x, event.y, 20)
  #canvas.create_line((lastx, lasty, event.x, event.y), width=20, tags="drawing")
  draw.line([event.x, event.y, lastx, lasty], fill=0, width=30)
  lastx, lasty = event.x, event.y
  currentStroke[0].append(event.x)
  currentStroke[1].append(event.y)
  currentStroke[2].append(time.time() - startTime)

def create_circle(x, y, r): #center coordinates, radius
  x0 = x - r
  y0 = y - r
  x1 = x + r
  y1 = y + r
  return canvas.create_oval(x0, y0, x1, y1, fill="black", tags="drawing")

# Called when the button is released on the canvas
def strokeEnd(event):
  global strokeData, currentStroke
  imageOutput = np.array(canvasImage)
  drawingStroke = False
  strokeData.append(currentStroke)
  currentStroke = []
  # createText(100, screeny - 50, "Stroke", 1, "TempText")
  predictAndCheck(3)
  #print(strokeData)

# Clears all drawings in the canvas
def clearCanvas():
  global strokeData, currentStroke
  canvas.delete("drawing")
  draw.rectangle([0, 0, screenx, screeny], 255)
  strokeData = []
  currentStroke = []

# Creates a temporary text on the screen that would be deleted in time
def createText(x=0, y=0, string="NullText", time = 2, tag = "TempText"):
  textCanvas.create_text(
      x, y, fill="darkblue",
      font="Times 20 italic bold",
      text=string,
      tags=tag)
  if time > 0:  
    threading.Timer(time, clearText).start()

# clears the text above 
def clearText():
  textCanvas.delete("TempText")

# Creates a prompt that stays on the canvas, deletes the old one if this is called again
def createPrompt(x=0, y=0, string="NullText"):
  textCanvas.delete("PromptText")
  textCanvas.create_text(
      x, y, fill="darkblue",
      font="Times 20 italic bold",
      text=string,
      tags="PromptText") 
  
# Continues the prompts in the game
def nextPrompt():
  global prompts, promptStartTime, promptTime, promptIndex, correctAnswers
  promptIndex += 1
  if promptIndex >= len(prompts):
    promptIndex += 1
  correctAnswers = []
  promptTime = 60
  promptStartTime = time.time()
  createPrompt(screenx/2, 50, prompts[promptIndex])
  textCanvas.delete("FoundAnswers")
  updateScore()

# Checks if the answer works for the prompt
def checkAnswers(answers):
  global promptAnswers, promptIndex, correctAnswers
  for answer in answers:
    print("checking", answer, answer in promptAnswers[promptIndex])
    if answer.lower() in promptAnswers[promptIndex]:
      # createText(screenx/2, screeny - offsety + 15, answer + " works!", 2)
      if answer not in correctAnswers:
        correctAnswers.append(answer)
        clearText()
        createText(screenx/2, screeny - offsety + 15, writeFeedback(answer), 4)
        updateScore()
        break

def writeFeedback(answer):
  global promptIndex
  feedback = answer + " works!"
  if promptIndex == 0 and answer == "hand":
    feedback = "Yeah, I guess a hand technically works"
  elif promptIndex == 0 and answer == "compass":
    feedback = "Compass! Yes! So you dont get lost!"
  elif promptIndex == 0 and answer == "square":
    feedback = "Paper works!"
  elif promptIndex == 0 and answer == "eye":
    feedback = "Definitely need a eyes for a lecture"
  elif promptIndex == 0 and answer == "eyeglasses":
    feedback = "Eyeglasses? Well, I guess if you need them"
  elif promptIndex == 1 and answer == "broom":
    feedback = "Is that a broom?"
  elif promptIndex == 1 and answer == "garden hose":
    feedback = "That's a garden hose! Definitely need that!"
  elif promptIndex == 1 and answer == "firetruck":
    feedback = "Firetruck! Right?"
  return feedback

# Updates the solution text of the current prompt
def updateScore():
  global promptAnswers, correctAnswers, promptIndex
  textCanvas.delete("Score")
  answersLeft = str(len(correctAnswers)) + "/" + str(len(promptAnswers[promptIndex]))
  print(answersLeft)
  # createText(100, screenY - 200, "Solutions Found: "+answersLeft, 0, "Score")
  createText(150, screeny - 200, "Solutions Found: "+answersLeft, 0, "Score")
  textCanvas.delete("FoundAnswers")
  index = 0
  for answers in correctAnswers:
    createText(screenx - 200, offsety + 100 + index * 50, getAnswer(answers), 0, "FoundAnswers")
    index += 1

def getAnswer(answer):
  global promptIndex
  feedback = answer
  if promptIndex == 0 and answer == "square":
    feedback = "paper"
  return feedback

def updateTimer():
  global promptStartTime, promptTime
  textCanvas.delete("Timer")
  timeLeft = int(promptTime - (time.time() - promptStartTime))
  if timeLeft <= 0:
    nextPrompt()
  createText(screenx - 100, 50, "Time: "+str(timeLeft), 0, "Timer")
  threading.Timer(1, updateTimer).start()
#Root Create + Setup
root = Tk()
root.geometry("1290x730")

#Canvas Create + Setup

textCanvas = Canvas(root, width=screenx, height=screeny)
textCanvas.place(x=0 ,y=0)

#Generate the border of the canvas

def predictAndCheck(top_n):
  print("testing")
  item = np.array(canvasImage)
  item = 255-item
  item_class = 0 # just so not broken
  print(np.shape(canvasImage))
  preds, probs = keras_predict(model, item)
  # print(preds)
  # pred_classes = [label_dict[i] for i in range(top_k)]
  pred_classes = []
  for idx in preds:
    # print(label_dict[idx])
    pred_classes.append(label_dict[idx])
  print("top labels:", pred_classes)
  pred_class = preds[0]
  print("prob:", probs[pred_class], "label:", pred_class)
  predictions = pred_classes[:top_n]
  if probs[pred_class] <= .2:
    clearText()
    createText(screenx/2, screeny - offsety + 15, "I don't know what this is", 2)
    return
  # checkAnswers(["hose"])
  checkAnswers(predictions)


canvas = Canvas(root, width=screenx- offsetx * 2, height=screeny- offsety* 2)
canvas.place(x=offsetx,y=offsety)
#canvas.grid(column=0, row=0, sticky=(N, W, E, S))
canvas.bind("<Button-1>", xy)
canvas.bind("<B1-Motion>", addLine)
canvas.bind("<ButtonRelease-1>", strokeEnd)
canvas.create_rectangle(8,8,screenx - offsetx * 2,screeny - offsety * 2,width=10)


# Create the clear button
B = Button(root, text ="Clear", command = clearCanvas)
buttonsize = [100, 50]
B.place(bordermode=OUTSIDE, height=buttonsize[1], width=buttonsize[0], x = (screenx - buttonsize[0]) / 2, y=screeny - buttonsize[1])

# Generate the prompt
promptStartTime = time.time()
createPrompt(screenx/2, 50, prompts[promptIndex])

updateTimer()
createText(150, screeny - 200, "Solutions Found: 0/"+str(len(promptAnswers[promptIndex])), 0, "Score")

# checkAnswers(predictions)
# predictAndCheck(5)

#Main Loop
root.mainloop()

In [ ]:
canvasImage.save("test.png")
imageOutput = np.array(canvasImage)
print(0 in imageOutput)
test = Image.fromarray(imageOutput)
test.save("arrayTest.png")

In [ ]:
def createTest():
  createPrompt(offsetx + screenx/2, 50, "test")

Load model and setup prediction functions requirements

In [ ]:
!pip install scikit-learn scipy matplotlib sklearn opencv-python tensorflow==2.1.0 tensorflow-estimator==2.1.0
# !conda install scikit-learn scipy matplotlib sklearn opencv tensorflow tensorflow-estimator=2.1.0

In [ ]:
import os
import ast
import pickle
# import pandas as pd
import numpy as np
# import seaborn as sns
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# from sklearn.utils import shuffle
from pathlib import Path

# from sklearn.model_selection import train_test_split
# from sklearn.utils import shuffle
# from keras.layers import Dense,Flatten, Conv2D
# from keras.layers import MaxPooling2D, Dropout
# from keras.utils import np_utils, print_summary
from keras.utils import np_utils
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
from keras.callbacks import TensorBoard
import cv2

In [ ]:
# Load label dict from pickle
path = r"C:\Users\kenny\OneDrive\Desktop\ScribbleSolver\label_dict.pickle"
#path = r"C:\Users\micha\Desktop\CMPM146\ScribbleSolver\Models\label_dict.pickle"
with open(path, 'rb') as handle:
    label_dict = pickle.load(handle)

In [ ]:
# model_save_dir = "/content/drive/My Drive/CMPM146 Final Project/Models/QuickDraw2.h5"
#model_save_dir = r"C:\Users\micha\Desktop\CMPM146\ScribbleSolver\Models\QuickDraw5.h5"
model_save_dir = r"C:\Users\kenny\OneDrive\Desktop\ScribbleSolver\QuickDraw5.h5"
model = tf.keras.models.load_model(model_save_dir)
model.summary()

In [ ]:
def keras_process_image(img):
    image_x = 28
    image_y = 28
    img = img / 255
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [ ]:
top_k = 10
def keras_predict(model, image):
    processed = keras_process_image(image)
    # print("processed: " + str(processed.shape))
    preds = model.predict(processed)[0]
    # pred_probab = model.predict(processed)[0]
    # topIndices = np.argpartition(preds, -1*top_k)[-1*top_k:]
    # topIndices[np.argsort(preds[ind])]
    topIndices = (-preds).argsort()[:top_k]
    # print("preds:", preds)
    # print("preds[0]", preds[0])
    # print("indices:", topIndices)
    # pred_class = list(pred_probab).index(max(pred_probab))
    # return max(pred_probab), pred_class
    # print(topIndices, preds)
    return topIndices, preds

In [ ]:
# Takes in a grayscale 28x28 image (or size 784) and returns 
# a pair label, class_num that is the models prediction for that input data
def predict(img):
  # load_model = tf.keras.models.load_model('QuickDraw.h5')
  load_model = tf.keras.models.load_model(model_save_dir)
  # load_model.summary()
  preds, probs = keras_predict(model, test_data)
  print(probs)
  pred_class = preds[0]
  # pred_probab, pred_class = keras_predict(model, test_data)
  print("Predicted class:", pred_class, "probability:", probs[pred_class])
  return pred_class, label_dict[pred_class]


In [ ]:
# test_item, test_item_class = get_item('fire hydrant')
test_item = np.array(canvasImage)
test_item = 255-test_item
test_item_class = 0 # just so not broken
print(np.shape(canvasImage))
# print(test_item_class, label_dict[test_item_class])
# test_img = np.reshape(test_item, (28, 28))
# print(np.shape(test_img))
# matplotlib.image.imsave('test.png', test_img)
matplotlib.pyplot.imshow(test_item, cmap='binary')
%time preds, probs = keras_predict(model, test_item)
# pred_classes = [label_dict[i] for i in range(top_k)]
pred_classes = []
for idx in preds:
  # print(label_dict[idx])
  pred_classes.append(label_dict[idx])
print("top labels:", pred_classes)
pred_class = preds[0]
print("prob:", probs[pred_class], "label:", pred_class)
# print("Correct label was:", test_item_class, label_dict[test_item_class])
# print("Match: ", test_item_class == pred_class)
# pred_class, label = predict(test_data)
# print('Predicted label:', label + ', ', "class:", pred_class)
